# **Analyzing Used Car Listings on eBay Kleinanzeigen**

This is from a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle. This version is a sample of 50,000 data points used for simulation and practice.

The data dictionary provided with data is as follows:

* dateCrawled - When this ad was first crawled. All field-values are taken from this date.
* name - Name of the car.
* seller - Whether the seller is private or a dealer.
* offerType - The type of listing
* price - The price on the ad to sell the car.
* abtest - Whether the listing is included in an A/B test.
* vehicleType - The vehicle Type.
* yearOfRegistration - The year in which which year the car was first registered.
* gearbox - The transmission type.
* powerPS - The power of the car in PS.
* model - The car model name.
* kilometer - How many kilometers the car has driven.
* monthOfRegistration - The month in which which year the car was first registered.
* fuelType - What type of fuel the car uses.
* brand - The brand of the car.
* notRepairedDamage - If the car has a damage which is not yet repaired.
* dateCreated - The date on which the eBay listing was created.
* nrOfPictures - The number of pictures in the ad.
* postalCode - The postal code for the location of the vehicle.
* lastSeenOnline - When the crawler saw this ad last online.

In [1]:
import pandas as pd
import numpy as np

In [2]:
autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [3]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The dataset contains 20 columns, most of which are stored as strings. There are a few columns with null values, but no columns have more than ~20% null values. There are some columns that contain dates stored as strings.

We'll start by cleaning the column names to make the data easier to work with.

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


* Change the columns from camelcase to snakecase.
* Change a few wordings to more accurately describe the columns.

## **Initial Data Exploration and Cleaning** 

Explore the data to find areas that need cleaning.

In [6]:
autos.describe(include = "all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 19:48:02,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Text columns where all (or nearly all) of the values are the same:
* seller
* offer_type

The nr_of_pictures column looks odd, we'll need to investigate this further.

In [7]:
autos["nr_of_pictures"].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

This does not look like useful information. Time to drop these three columns.

In [8]:
autos = autos.drop(["nr_of_pictures", "seller", "offer_type"], axis=1)

Clean and convert price and odometer columns:

In [9]:
autos["price"] = (autos["price"]
                          .str.replace("$","")
                          .str.replace(",","")
                          .astype(int)
                          )
autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [10]:
autos["odometer"] = (autos["odometer"]
                            .str.replace("km","")
                            .str.replace(",","")
                            .astype(int)
                            )
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
autos["odometer_km"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

## **Exploring Odometer and Price**

In [11]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head()

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
Name: price, dtype: int64

In [12]:
autos["price"].value_counts().sort_index(ascending = True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

In [13]:
autos["price"].value_counts().sort_index(ascending = False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

At first glance, it may seem like there are two outliers on the low end. However, it is common for prices to start at 1 for bidding. We'll remove the 0 ones. Also, the few that are over 350,000

In [14]:
autos = autos[autos["price"].between(1,350000)]
autos.describe()

,price,registration_year,power_ps,odometer_km,registration_month,postal_code
count,48565.000000,48565.000000,48565.000000,48565.000000,48565.000000,48565.000000
mean,5888.935591,2004.755421,117.197158,125770.101925,5.782251,50975.745207
std,9059.854754,88.643887,200.649618,39788.636804,3.685595,25746.968398
min,1.000000,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1200.000000,1999.000000,71.000000,125000.000000,3.000000,30657.000000
50%,3000.000000,2004.000000,107.000000,150000.000000,6.000000,49716.000000
75%,7490.000000,2008.000000,150.000000,150000.000000,9.000000,71665.000000
max,350000.000000,9999.000000,17700.000000,150000.000000,12.000000,99998.000000


In [15]:
print(autos["odometer_km"].unique().shape)
print(autos["odometer_km"].describe())
autos["odometer_km"].value_counts().head()

(13,)
count     48565.000000
mean     125770.101925
std       39788.636804
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
Name: odometer_km, dtype: int64

In [16]:
autos["odometer_km"].value_counts().sort_index(ascending = True).head(13)

5000        836
10000       253
20000       762
30000       780
40000       815
50000      1012
60000      1155
70000      1217
80000      1415
90000      1734
100000     2115
125000     5057
150000    31414
Name: odometer_km, dtype: int64

Odometer data looks good from here. We can also use the int values to extract dates coming up.
## Exploring the date columns
Columns with date values:
* date_crawled
* registration_month
* registration_year
* ad_created
* last_seen

In [17]:
autos[["date_crawled","ad_created","last_seen"]][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [18]:
(autos["date_crawled"]
            .str[:10]
            .value_counts(normalize = True, dropna = False)
            .sort_index(ascending = True)
            )

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [19]:
(autos["ad_created"]
            .str[:10]
            .value_counts(normalize = True, dropna = False)
            .sort_index(ascending = True)
            )

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [20]:
(autos["last_seen"]
            .str[:10]
            .value_counts(normalize = True, dropna = False)
            .sort_index(ascending = True)
            )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

Findings: The date_crawled column looks good and has no clear bad data. The ad_created column looks good too but there are a lot of values - spanning as far back as 9 months. The last_seen column has disproportionate data for the last three days - probably more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [21]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Looking at this column, we have some clear false information. The lowest value is 1000, while the highest is 9999, not valid years for cars.
## Dealing with Incorrect Registration Year Data

In [22]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.038793369710697

This data is from 2016, so we can confidently say any car not between 1900-2016 is probably false data. Here we see about 3.9% of the data falls outside of this range. We can remove these.

In [23]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize = True)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
2008    0.047450
2009    0.044665
1997    0.041794
2011    0.034768
2010    0.034040
1996    0.029412
2012    0.028063
1995    0.026285
2016    0.026135
2013    0.017202
2014    0.014203
1994    0.013474
1993    0.009104
2015    0.008397
1992    0.007926
1990    0.007433
1991    0.007262
1989    0.003727
1988    0.002892
1985    0.002035
          ...   
1966    0.000471
1976    0.000450
1969    0.000407
1975    0.000386
1965    0.000364
1964    0.000257
1963    0.000171
1959    0.000129
1961    0.000129
1910    0.000107
1956    0.000086
1958    0.000086
1937    0.000086
1962    0.000086
1950    0.000064
1954    0.000043
1941    0.000043
1951    0.000043
1934    0.000043
1957    0.000043
1955    0.000043
1953    0.000021
1943    0.000021
1929    0.000021
1939    0.000021
1938    0.000021
1948    0.000021
1927    0.0000

Now the data for this column looks correct, with top-heavy year counts as expected.
## Exploring Price by Brand

In [24]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

There are a lot of brands, so we limit ourselves to the brands with over 5% of the total listings.

In [25]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [26]:
brand_mean_prices = {}
for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)
    
brand_mean_prices

{'audi': 9336,
 'bmw': 8332,
 'ford': 3749,
 'mercedes_benz': 8628,
 'opel': 2975,
 'volkswagen': 5402}

We see that Audi, BMW, Mercedes are more expensive (as expected), ford and opel are less expensive, and volkswagen is in the middle. It is also the most popular brand interestingly, which makes the argument that people will go more for a middle tier, best of both worlds price. However, it could also be mileage.
## Exploring Mileage

We now create a dataframe that can easily look at mean prices compared to mean mileage as well.

In [27]:
brand_mean_mileage = {}
for brand in common_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

In [28]:
brand_info = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
brand_info

,mean_mileage
bmw,132572
mercedes_benz,130788
opel,129310
audi,129157
volkswagen,128707
ford,124266


In [29]:
brand_info["mean_price"] = mean_prices
brand_info

,mean_mileage,mean_price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage. Overall though, the assumption made earlier based on prices alone seems to hold better weight in terms of listings.

## More Data Cleaning:
## Translating German Words

In [30]:
autos.head(10)

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,7900,test,bus,2006,automatik,150,voyager,150000,4,diesel,chrysler,NaN,2016-03-21 00:00:00,22962,2016-04-06 09:45:21
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,300,test,limousine,1995,manuell,90,golf,150000,8,benzin,volkswagen,NaN,2016-03-20 00:00:00,31535,2016-03-23 02:48:59
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,1990,control,limousine,1998,manuell,90,golf,150000,12,diesel,volkswagen,nein,2016-03-16 00:00:00,53474,2016-04-07 03:17:32
8,2016-03-22 16:51:34,Seat_Arosa,250,test,NaN,2000,manuell,0,arosa,150000,10,NaN,seat,nein,2016-03-22 00:00:00,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,590,control,bus,1997,manuell,90,megane,150000,7,benzin,renault,nein,2016-03-16 00:00:00,15749,2016-04-06 10:46:35


It seems the columns "gearbox" and "unrepaired_damage" can be translated to English.

In [31]:
autos["gearbox"].value_counts()

manuell      34715
automatik     9856
Name: gearbox, dtype: int64

In [32]:
mapping_gearbox = {
    'manuell': 'manual',
    'automatik': 'automatic'
}

autos["gearbox"] = autos["gearbox"].map(mapping_gearbox)
autos["gearbox"].value_counts()

manual       34715
automatic     9856
Name: gearbox, dtype: int64

In [33]:
autos["unrepaired_damage"].value_counts()

nein    33834
ja       4540
Name: unrepaired_damage, dtype: int64

In [34]:
mapping_damage = {
    'nein': 'no',
    'ja': 'yes'
}

autos["unrepaired_damage"] = autos["unrepaired_damage"].map(mapping_damage)
autos["unrepaired_damage"].value_counts()

no     33834
yes     4540
Name: unrepaired_damage, dtype: int64

## Converting Numeric Dates from Data
The 2 regiser dates were already in int format, so we look at the other 3 previously mentioned. We will convert these to integers as well.

In [35]:
autos["date_crawled"] = (autos["date_crawled"]
                                     .str.split(n = 1, expand = True)
                                     .iloc[:,0]
                                     .str.replace("-","")
                                     .astype(int)
                        )

autos["ad_created"] = (autos["ad_created"]
                                     .str.split(n = 1, expand = True)
                                     .iloc[:,0]
                                     .str.replace("-","")
                                     .astype(int)
                      )

autos["last_seen"] = (autos["last_seen"]
                                     .str.split(n = 1, expand = True)
                                     .iloc[:,0]
                                     .str.replace("-","")
                                     .astype(int)
                     )

In [36]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manual,158,andere,150000,3,lpg,peugeot,no,20160326,79588,20160406
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatic,286,7er,150000,6,benzin,bmw,no,20160404,71034,20160406
2,20160326,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manual,102,golf,70000,7,benzin,volkswagen,no,20160326,35394,20160406
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatic,71,fortwo,70000,6,benzin,smart,no,20160312,33729,20160315
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manual,0,focus,150000,7,benzin,ford,no,20160401,39218,20160401


## Extracting the Name Column
In the names column, it looks like we can extract the make from the name itself. The name is already in the brand, so we can drop this and are left with the make of the car, supplementing the model column. We can probaly extract more columns, but I do not know enough about the car information provided in the string to know what is relevant information and what is not.

In [37]:
autos["name"] = (autos["name"]
                         .str.partition("_")
                         .iloc[:,2]
                )

autos.rename({"name": "make"}, axis=1, inplace=True)

In [38]:
autos.head(10)

,date_crawled,make,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,20160326,807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manual,158,andere,150000,3,lpg,peugeot,no,20160326,79588,20160406
1,20160404,740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatic,286,7er,150000,6,benzin,bmw,no,20160404,71034,20160406
2,20160326,Golf_1.6_United,8990,test,limousine,2009,manual,102,golf,70000,7,benzin,volkswagen,no,20160326,35394,20160406
3,20160312,smart_fortwo_coupe_softouch/F1/Klima/Panorama,4350,control,kleinwagen,2007,automatic,71,fortwo,70000,6,benzin,smart,no,20160312,33729,20160315
4,20160401,Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepflegt.mit...,1350,test,kombi,2003,manual,0,focus,150000,7,benzin,ford,no,20160401,39218,20160401
5,20160321,Grand_Voyager_2.8_CRD_Aut.Limited_Stow´n_Go_Si...,7900,test,bus,2006,automatic,150,voyager,150000,4,diesel,chrysler,NaN,20160321,22962,20160406
6,20160320,Golf_III_GT_Special_Electronic_Green_Metallic_...,300,test,limousine,1995,manual,90,golf,150000,8,benzin,volkswagen,NaN,20160320,31535,20160323
7,20160316,IV_1.9_TDI_90PS,1990,control,limousine,1998,manual,90,golf,150000,12,diesel,volkswagen,no,20160316,53474,20160407
8,20160322,Arosa,250,test,NaN,2000,manual,0,arosa,150000,10,NaN,seat,no,20160322,7426,20160326
9,20160316,Megane_Scenic_1.6e_RT_Klimaanlage,590,control,bus,1997,manual,90,megane,150000,7,benzin,renault,no,20160316,15749,20160406


## More Analysis:
## Most Common Brand/Model Combinations
To the best of my knowledge, a model seems to be unique to a brand (it doesn't seem like and also doesn't make sense for a brand to copy another brand's model name), so we will assume this.

In [39]:
model_counts = autos["model"].value_counts(normalize=True)
common_models = model_counts[model_counts > .03].index
common_models

Index(['golf', 'andere', '3er', 'polo', 'corsa', 'passat', 'astra'], dtype='object')

In [40]:
autos["model"].value_counts(normalize = True).head(7)

golf      0.083326
andere    0.075818
3er       0.058780
polo      0.036167
corsa     0.035785
passat    0.030323
astra     0.030300
Name: model, dtype: float64

In [41]:
for model in common_models:
    model_only = autos[autos["model"] == model]
    brand = model_only["brand"].iloc[0]
    count = model_only["model"].size
    print("\n" + model + "\n")
    print(model_only["brand"].head(10))


golf

2      volkswagen
6      volkswagen
7      volkswagen
38     volkswagen
40     volkswagen
58     volkswagen
62     volkswagen
96     volkswagen
120    volkswagen
151    volkswagen
Name: brand, dtype: object

andere

0      peugeot
19       mazda
45      toyota
52        opel
70        saab
75       volvo
95      jaguar
97       mazda
136     subaru
144      skoda
Name: brand, dtype: object

3er

18     bmw
76     bmw
82     bmw
203    bmw
270    bmw
300    bmw
309    bmw
336    bmw
338    bmw
364    bmw
Name: brand, dtype: object

polo

26     volkswagen
42     volkswagen
112    volkswagen
127    volkswagen
147    volkswagen
148    volkswagen
187    volkswagen
374    volkswagen
448    volkswagen
458    volkswagen
Name: brand, dtype: object

corsa

32     opel
66     opel
116    opel
158    opel
221    opel
265    opel
279    opel
282    opel
379    opel
405    opel
Name: brand, dtype: object

passat

57     volkswagen
114    volkswagen
124    volkswagen
231    volkswagen
256    

Upon further investigation, we find that indeed the top models match up pretty well with the top brands, with BMW, Volkswagen, and Opel holding all of the top models, which was expected snice they were the top three brands. There was one exception, however: andere. Andere means "other" in German, so it makes sense now that there are a bunch of brands inside of this "model" type. Here we see andere expanded:

In [42]:
andere_only = autos[autos["model"] == "andere"]
andere_only["brand"].value_counts()

mercedes_benz    439
fiat             229
citroen          219
audi             216
ford             190
opel             148
chevrolet        147
peugeot          146
hyundai          146
mazda            138
nissan           121
suzuki           119
renault          117
toyota           106
mitsubishi        97
volkswagen        96
volvo             82
kia               65
alfa_romeo        60
rover             55
saab              53
chrysler          52
honda             51
porsche           46
jaguar            39
bmw               38
skoda             36
seat              23
jeep              15
daihatsu          14
mini              12
lancia            11
subaru            10
daewoo             9
trabant            8
smart              7
lada               7
land_rover         4
dacia              2
Name: brand, dtype: int64

We see that Mercedes is actually holding a big portion of the andere models. Even though andere was the 2nd leading model, after breaking it down, we can see that it is in fact not an actual model and so not as common as the others. We conclude that the top 5 brand/model combinations are:
* Volkswagen Golf
* BMW 3er
* Volkswagen Polo
* Opel Corsa
* Volkswagen Passat


## odometer_km Patterns

We split the mileages into 3 groups: under 50000, 50000-100000, and over 100000. We then use these groups to determine any patterns in average prices based on mileage.

In [43]:
autos["odometer_km"].value_counts().sort_index()

5000        785
10000       241
20000       742
30000       760
40000       797
50000       993
60000      1128
70000      1187
80000      1375
90000      1673
100000     2058
125000     4857
150000    30085
Name: odometer_km, dtype: int64

In [44]:
below_50 = autos[autos["odometer_km"] < 50000]
between_50_100 = autos[(autos["odometer_km"] > 50000) & (autos["odometer_km"] < 100000)]
over_100 = autos[autos["odometer_km"] > 100000]

below_50_price = below_50["price"].mean()
between_50_100_price = between_50_100["price"].mean()
over_100_price = over_100["price"].mean()

print("Below 50K miles price: " + str(below_50_price))
print("Between 50K-100K miles price: " + str(between_50_100_price))
print("Over 100K miles price: " + str(over_100_price))

Below 50K miles price: 15212.965714285714
Between 50K-100K miles price: 10156.72496736901
Over 100K miles price: 4107.938641176807


As expected, there is what seems like a linear drop in price as mileage goes up. This is not surprising and we can be confident the data is correct.

## Damaged vs. Non-Damaged
Lastly, we look at prices betwen damaged and non-damaged cars in the dataset. We'll ignore null values because we cannot quantify damaged or not and put it in a price, although theoretically the null price should be between the two.

In [45]:
print(autos["unrepaired_damage"].describe())
autos["unrepaired_damage"].value_counts(dropna = False)

count     38374
unique        2
top          no
freq      33834
Name: unrepaired_damage, dtype: object


no     33834
NaN     8307
yes     4540
Name: unrepaired_damage, dtype: int64

In [46]:
damaged_cars = autos[autos["unrepaired_damage"] == "yes"]
not_damaged_cars = autos[autos["unrepaired_damage"] == "no"]
undet_cars = autos[autos["unrepaired_damage"].isnull()]

damaged_cars_price = damaged_cars["price"].mean()
not_damaged_cars_price = not_damaged_cars["price"].mean()
undet_cars_price = undet_cars["price"].mean()

print("Damaged car price: " + str(damaged_cars_price))
print("Not Damaged/Repaired car price: " + str(not_damaged_cars_price))
print("Unknown car price: " + str(undet_cars_price))

Damaged car price: 2241.146035242291
Not Damaged/Repaired car price: 7164.033102796004
Unknown car price: 3188.046105693993


As we can see, not damaged or repaired cars costs over 3 times more than their damaged or unrepaired counterparts. This makes sense after all. Also, our theory of the unknown cars being in the middle also turned out to be true, as expected, as it could have some members from each category. These cars can 